##620. Not Boring Movies
### Table: Cinema

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| movie       | varchar |
| description | varchar |
| rating      | float   |

**Primary Key**: `id` — each row contains unique information about a movie.

**Description**:
Each row contains the name of a movie, its description (genre or comment), and its rating.  
The `rating` is a float with 2 decimal places in the range [0, 10].

---

### 🧠 Business Logic:
Report the movies that meet **both** of the following criteria:
- Have an **odd-numbered ID**
- Have a **description that is not "boring"**

Return the result table **ordered by rating in descending order**.

---

### ✅ Expected Output Format:

#### Input:
| id | movie      | description | rating |
|----|------------|-------------|--------|
| 1  | War        | great 3D    | 8.9    |
| 2  | Science    | fiction     | 8.5    |
| 3  | irish      | boring      | 6.2    |
| 4  | Ice song   | Fantacy     | 8.6    |
| 5  | House card | Interesting | 9.1    |

#### Output:
| id | movie      | description | rating |
|----|------------|-------------|--------|
| 5  | House card | Interesting | 9.1    |
| 1  | War        | great 3D    | 8.9    |

**Explanation**:  
Odd IDs are 1, 3, and 5.  
Movie with ID = 3 is "boring" → excluded.  
Movies with ID = 1 and 5 are included and sorted by rating.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

# Sample data
data = [
    (1, "War", "great 3D", 8.9),
    (2, "Science", "fiction", 8.5),
    (3, "irish", "boring", 6.2),
    (4, "Ice song", "Fantacy", 8.6),
    (5, "House card", "Interesting", 9.1)
]

# Define schema
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("movie", StringType(), True),
    StructField("description", StringType(), True),
    StructField("rating", FloatType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Register Temp View
df.createOrReplaceTempView("Cinema")

# SQL logic to filter odd IDs and non-boring descriptions
result = spark.sql("""
    SELECT id, movie, description, rating
    FROM Cinema
    WHERE id % 2 = 1 AND description != 'boring'
    ORDER BY rating DESC
""")

# Display result
display(result)

In [0]:
%sql
Select id , movie , description , CAST(rating AS DECIMAL(4,1))  as rating from cinema where id%2 = 1 and description <> 'boring' order by rating desc 

In [0]:
from pyspark.sql.functions import *

df.filter((col("id") % 2 != 0) & (col("description") != "boring"))\
    .selectExpr("id", "movie", "description", "cast(rating as decimal(4,1)) as rating")\
    .orderBy(col("rating").desc())\
    .display()